In [8]:
from PIL import Image
import io

In [9]:
def encode_image_to_binary(image_path, output_binary_path, format="JPEG"):
    try:
        # Open the image file
        with Image.open(image_path) as img:
            # Create an in-memory bytes buffer
            img_byte_arr = io.BytesIO()
            
            # Save the image into the buffer in the specified format (JPEG, PNG, etc.)
            img.save(img_byte_arr, format=format)
            
            # Get the binary data from the buffer
            img_binary_data = img_byte_arr.getvalue()

        # Write the binary data to a file
        with open(output_binary_path, 'wb') as binary_file:
            binary_file.write(img_binary_data)

        print(f"Image encoded and saved as binary to {output_binary_path}")

    except Exception as e:
        print(f"Error encoding image: {e}")

In [10]:
image_path = "download.jpeg"
output_binary_path = "encoded_image.bin"
format = "JPEG"

encode_image_to_binary(image_path, output_binary_path, format)

Image encoded and saved as binary to encoded_image.bin


In [11]:
def bin_to_coe_128bit(bin_filename, coe_filename):
    # Read the binary file
    with open(bin_filename, "rb") as bin_file:
        binary_data = bin_file.read()
    
    # Convert binary data into 128-bit chunks
    hex_data = []
    for i in range(0, len(binary_data), 16):  # Process 16 bytes (128 bits) at a time
        chunk = binary_data[i:i+16]
        
        # Ensure chunk is 16 bytes by padding if needed
        if len(chunk) < 16:
            chunk = chunk + bytes(16 - len(chunk))
        
        # Convert each 16-byte chunk to a single 128-bit hex string
        hex_string = ''.join(f"{byte:02X}" for byte in chunk)
        hex_data.append(hex_string)

    # Write data to COE file
    with open(coe_filename, "w") as coe_file:
        coe_file.write("memory_initialization_radix = 16;\n")
        coe_file.write("memory_initialization_vector = \n")
        coe_file.write(",\n".join(hex_data))
        coe_file.write(";")  # End with a semicolon for COE format

    print(f"COE file '{coe_filename}' generated successfully.")


In [12]:
bin_filename = "encoded_image.bin"  # Replace with your .bin file path
coe_filename = "image128.coe"  # Replace with desired .coe file path
bin_to_coe_128bit(bin_filename, coe_filename)

COE file 'image128.coe' generated successfully.


In [13]:
def split_coe_file(input_file, output_files):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Initialize variables to locate radix and vector start points
    radix_line = None
    data_start = None
    
    # Locate radix and data start lines, ignoring case and whitespace
    for i, line in enumerate(lines):
        line_clean = line.lower().replace(" ", "")
        if "memory_initialization_radix=" in line_clean:
            radix_line = line.strip()
        elif "memory_initialization_vector=" in line_clean:
            data_start = i + 1
            break
    
    # Check if both radix and data start lines were found
    if data_start is None or radix_line is None:
        print("Error: Could not find necessary lines in the .coe file.")
        return

    # Extract the data lines and clean up
    data_lines = []
    for line in lines[data_start:]:
        cleaned_line = line.strip().rstrip(";")  # Remove whitespace and trailing semicolons
        if cleaned_line:
            data_lines.extend(cleaned_line.split(","))  # Split comma-separated values

    # Split data among the output files
    num_outputs = len(output_files)
    split_data = [[] for _ in range(num_outputs)]
    for i, value in enumerate(data_lines):
        split_data[i % num_outputs].append(value.strip())

    # Write each split data to corresponding output .coe file
    for idx, out_file in enumerate(output_files):
        with open(out_file, 'w') as outfile:
            # Write the header with the same radix as in the input file
            outfile.write(f"{radix_line}\n")
            outfile.write("memory_initialization_vector=\n")
            # Write the data, adding commas only between values, and ending with a semicolon
            outfile.write(",\n".join(split_data[idx]) + ";\n")

    print(f"Successfully split {input_file} into {len(output_files)} files.")


In [14]:
input_file = "image128.coe"
output_files = ["output_file1.coe", "output_file2.coe", "output_file3.coe"]
split_coe_file(input_file, output_files)

Successfully split image128.coe into 3 files.
